# Implementation of Xception Model
*by Marvin Bertin*
<img src="../../images/keras-tensorflow-logo.jpg" width="400">

# Imports

In [15]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

In [16]:
models = tf.contrib.keras.models
layers = tf.contrib.keras.layers
initializers = tf.contrib.keras.initializers
regularizers = tf.contrib.keras.regularizers

# Xception Model

<img src="../../images/xception0.png" width="800">

## Convolution Block

In [17]:
def conv_block(x, filters, block_num, conv_num, strides=(1,1)):
    name = 'block{}_conv{}_'.format(block_num, conv_num)
    
    # conv-BN-relu
    x = layers.Conv2D(filters, (3,3), strides=(2,2), use_bias=False, name=name)(x)
    x = layers.BatchNormalization(name=name+'bn')(x)
    x = layers.Activation('relu', name=name+'act')(x)

    return x

## Separable Convolution Block

<img src="../../images/xception1.jpg" width="500">


In [4]:
def separable_conv_block(x, filters, block_num, conv_num, pre_activation=None):
    name = 'block{}_sepconv{}_'.format(block_num, conv_num)

    if pre_activation is True:
        x = layers.Activation('relu', name=name+'act')(x)
        
    # (relu)-sepconv-BN-(relu)
    x = layers.SeparableConv2D(filters, (3,3), padding='same', use_bias=False, name=name)(x)
    x = layers.BatchNormalization(name=name+'bn')(x)
    
    if pre_activation is False:
        x = layers.Activation('relu', name=name+'act')(x)
    

    return x

## Middle Flow Block

<img src="../../images/middle-flow.png" width="200">


In [5]:
def middle_flow_block(x, filters, block_num):

    # middle flow
    
    residual = x
    
    x = separable_conv_block(x, filters, block_num=block_num, conv_num='1', pre_activation=True)
    x = separable_conv_block(x, filters, block_num=block_num, conv_num='2', pre_activation=True)
    x = separable_conv_block(x, filters, block_num=block_num, conv_num='3', pre_activation=True)
    
    return layers.add([x, residual])

## Xception Block

<img src="../../images/xception-block1.png" width="400">


In [10]:
def xception_block(x, filters, block_num, pre_activation=True):
    block = 'block{}_'.format(block_num)
    filter_conv1, filter_conv2 = filters

    # residual conv branch
    residual = layers.Conv2D(filter_conv2, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
    residual = layers.BatchNormalization()(residual)

    # separable conv block
    x = separable_conv_block(x, filter_conv1, block_num=block_num, conv_num='1', pre_activation=pre_activation)
    x = separable_conv_block(x, filter_conv2, block_num=block_num, conv_num='2', pre_activation=True)

    # downsampling and merging
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name=block+'pool')(x)
    
    return layers.add([x, residual])

# Entry Flow

<img src="../../images/entry-flow.png" width="300">


In [ ]:
#===========ENTRY FLOW==============
#Block 1
x = conv_block(img_input, 32, block_num='1', conv_num='1', strides=(2,2))
x = conv_block(x, 64, block_num='1', conv_num='2')

#Block 2
x = xception_block(x, (128, 128), '2', pre_activation=False)

#Block 3
x = xception_block(x, (256, 256), '3')

#Block 4
x = xception_block(x, (728, 728), '4')

# Middle Flow

<img src="../../images/middle-flow.png" width="200">


In [ ]:
#===========MIDDLE FLOW===============
for i in range(8):
    block_num = str(5+i)
    x = middle_flow_block(x, 728, block_num)

# Exit Flow

<img src="../../images/exit-flow.png" width="300">


In [ ]:
#========EXIT FLOW============
#Block 13
x = xception_block(x, (728, 1024), '13') # second conv is different

# Block 14
x = separable_conv_block(x, 1536, block_num='14', conv_num='1', pre_activation=False)
x = separable_conv_block(x, 2048, block_num='14', conv_num='2', pre_activation=False)

# logistic regression
x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
x = layers.Dense(classes, activation='softmax', name='predictions')(x)

## Full Xception Model

In [7]:
def Xception(input_shape=(299,299,3), classes=1000):
    """Instantiates the Xception architecture.
    """
    
    img_input = layers.Input(shape=input_shape)

    #===========ENTRY FLOW==============
    #Block 1
    x = conv_block(img_input, 32, block_num='1', conv_num='1', strides=(2,2))
    x = conv_block(x, 64, block_num='1', conv_num='2')

    #Block 2
    x = xception_block(x, (128, 128), '2', pre_activation=False)

    #Block 3
    x = xception_block(x, (256, 256), '3')

    #Block 4
    x = xception_block(x, (728, 728), '4')
         

    #===========MIDDLE FLOW===============
    for i in range(8):
        block_num = str(5+i)
        x = middle_flow_block(x, 728, block_num)

    #========EXIT FLOW============
    #Block 13
    x = xception_block(x, (728, 1024), '13') # second conv is different

    # Block 14
    x = separable_conv_block(x, 1536, block_num='14', conv_num='1', pre_activation=False)
    x = separable_conv_block(x, 2048, block_num='14', conv_num='2', pre_activation=False)

    # logistic regression
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(classes, activation='softmax', name='predictions')(x)

    # Create model.
    model = models.Model(inputs=img_input, outputs=x, name='xception')
    return model

In [8]:
xception = Xception()
xception.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1_ (Conv2D)           (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1_[0][0]              
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

## Next Lesson
### Train and Evaluate Xception
- Image classification task with ImageNet dataset

<img src="../../images/divider.png" width="100">